In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from causal_nh.utils import open_pkl_file
from causal_nh.test import test_prediction
import os
from causal_nh.train import train_nh, train_causal_nh
from causal_nh.model.NeuralHawkes import CausalNeuralHawkesMasked, CausalNeuralHawkesMaskedWeighted, CausalNeuralHawkesTrainableWeighted
import pickle
%matplotlib inline

In [7]:
data_path = "../../data"
results = "../../results"
artifacts = "../../artefacts"
if not os.path.exists(results):
    os.mkdir(results)
if not os.path.exists(artifacts):
    os.mkdir(artifacts)

datasets = [item for item in os.listdir(data_path) if os.path.isdir(f'{data_path}/{item}')]
print(datasets)

['data_synth_10_events_small', 'data_synth_2_events_small', 'data_synth_3_events_small', 'data_synth_5_events_small']


In [ ]:
new_models = [CausalNeuralHawkesMasked, CausalNeuralHawkesMaskedWeighted, CausalNeuralHawkesTrainableWeighted]
for dataset in datasets:
    dataset_name = dataset
    path = f'{data_path}/{dataset_name}/'
    print(path)
    train = open_pkl_file(path + 'train.pkl', 'train')
    dev = open_pkl_file(path + 'dev.pkl', 'dev')
    test_data = open_pkl_file(path + 'test.pkl', 'test')
    with open(path + 'A.pkl', 'rb') as f:
        A = pickle.load(f)
    A = A['A']
    W = A
    A[A==0.2] = 1

    output_folder = f'{artifacts}/{dataset_name}'
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)
    
    save_path, model_path = train_nh(train, dev, f'{output_folder}/', False, lr=0.01, num_epochs=100, batch_size=50)
    
    original_time, original_type, estimated_times, estimated_types, estimated_intensities, rmse, acc = test_prediction(
                test_data = test_data,
                idx_test_series=5,
                n_samples=1000,
                start_test_idx=10,
                path_to_save=save_path,
                path_to_model=model_path,
                dataset_name=dataset_name)
    
    fname = save_path+model_path.split('/')[-1].split('.')[0]+"_metrics.txt"
    with open(fname, 'w') as f:
        f.write(f"RMSE: {rmse}\n")
        f.write(f"Accuracy: {acc}")
    
    for m in new_models:
        print(m.__name__)
        save_path, model_path = train_causal_nh(train, dev, A,W,  f'{output_folder}/', False, lr=0.01, num_epochs=100, batch_size=50, dataset_name=dataset_name, model=m)
        
        original_time, original_type, estimated_times, estimated_types, estimated_intensities, rmse, acc = test_prediction(
                test_data = test_data,
                idx_test_series=5,
                n_samples=1000,
                start_test_idx=10,
                path_to_save=save_path,
                path_to_model=model_path,
                dataset_name=dataset_name)
        fname = save_path+model_path.split('/')[-1].split('.')[0]+"_metrics.txt"
        with open(fname, 'w') as f:
            f.write(f"RMSE: {rmse}\n")
            f.write(f"Accuracy: {acc}")

../../data/data_synth_10_events_small/
Processing data...
You are using GPU acceleration.
Number of GPU:  6
start training...
Epoch 0, process 0 out of 6 is done
Epoch 0, process 1 out of 6 is done
Epoch 0, process 2 out of 6 is done
Epoch 0, process 3 out of 6 is done
Epoch 0, process 4 out of 6 is done
Epoch 0, process 5 out of 6 is done
The log-likelihood at epoch 0: -0.13048886904625273

validating on log likelihood...

validating on type prediction accuracy if we know when next event will happen...


Epoch 1, process 0 out of 6 is done
Epoch 1, process 1 out of 6 is done
Epoch 1, process 2 out of 6 is done
Epoch 1, process 3 out of 6 is done
Epoch 1, process 4 out of 6 is done
Epoch 1, process 5 out of 6 is done
The log-likelihood at epoch 1: 0.8057503021444655

validating on log likelihood...

validating on type prediction accuracy if we know when next event will happen...


Epoch 2, process 0 out of 6 is done
Epoch 2, process 1 out of 6 is done
Epoch 2, process 2 out of 6 is don

Epoch 21, process 0 out of 6 is done
Epoch 21, process 1 out of 6 is done
Epoch 21, process 2 out of 6 is done
Epoch 21, process 3 out of 6 is done
Epoch 21, process 4 out of 6 is done
Epoch 21, process 5 out of 6 is done
The log-likelihood at epoch 21: 1.8727227046921202

validating on log likelihood...

validating on type prediction accuracy if we know when next event will happen...


Epoch 22, process 0 out of 6 is done
Epoch 22, process 1 out of 6 is done
Epoch 22, process 2 out of 6 is done
Epoch 22, process 3 out of 6 is done
Epoch 22, process 4 out of 6 is done
Epoch 22, process 5 out of 6 is done
The log-likelihood at epoch 22: 1.87523559548213

validating on log likelihood...

validating on type prediction accuracy if we know when next event will happen...


Epoch 23, process 0 out of 6 is done
Epoch 23, process 1 out of 6 is done
Epoch 23, process 2 out of 6 is done
Epoch 23, process 3 out of 6 is done
Epoch 23, process 4 out of 6 is done
Epoch 23, process 5 out of 6 is done
